## Local Search

In [ ]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [56]:
INPUT_DIR = r"C:\Users\aamruthvenkatesh.s\Music\GraphRag"
LANCEDB_URI =  r"C:\Users\aamruthvenkatesh.s\Music\GraphRag\lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [57]:
entity_df = pd.read_parquet(r'C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_nodes.parquet')
entity_embedding_df = pd.read_parquet(r"C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_entities.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 2624


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,MINISTRY OF LAW AND JUSTICE,ORGANIZATION,The Ministry of Law and Justice is a governmen...,ecefbf6e5a5967b21e19fc00d93af357,6,2,0,b45241d70f0e43fca764df95b2b81f77,2,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,NEW DELHI,GEO,The capital city of India where the Ministry o...,ecefbf6e5a5967b21e19fc00d93af357,6,3,1,4119fd06010c494caa07f439b333f4c5,3,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,PARLIAMENT,ORGANIZATION,The supreme legislative body of India that ena...,ecefbf6e5a5967b21e19fc00d93af357,6,5,2,d3835bf3dda84ead99deadbeac5d0d7d,5,None,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,PRESIDENT,PERSON,The President of India is the head of state an...,"0539816d48ef7448d37701037030e1fc,ae6df3c03fa2d...",6,23,3,077d2820ae1845bcbb1803379a3d1eae,23,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,"BHARATIYA NAGARIK SURAKSHA SANHITA, 2023",EVENT,An Act to consolidate and amend the law relati...,ecefbf6e5a5967b21e19fc00d93af357,6,3,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,3,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


In [58]:
relationship_df = pd.read_parquet(r'C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_relationships.parquet')
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 1224


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,MINISTRY OF LAW AND JUSTICE,PARLIAMENT,1.0,The Ministry of Law and Justice is responsible...,[ecefbf6e5a5967b21e19fc00d93af357],58a8fa7f29e347bdb9689b70b065a779,0,2,5,7
1,MINISTRY OF LAW AND JUSTICE,NEW DELHI,1.0,The Ministry of Law and Justice is located in ...,[ecefbf6e5a5967b21e19fc00d93af357],fae3fe31deb141ab93143ac411f1eaaa,1,2,3,5
2,NEW DELHI,PARLIAMENT,1.0,Parliament is located in New Delhi,[ecefbf6e5a5967b21e19fc00d93af357],a2cb46c226b94831853a5d28c5d94b0a,2,3,5,8
3,NEW DELHI,PRESIDENT,1.0,The President's office is located in New Delhi,[ecefbf6e5a5967b21e19fc00d93af357],d3511ecd27cd4166bdb39e757e275300,3,3,23,26
4,PARLIAMENT,PRESIDENT,1.0,The President gave assent to the Bharatiya Nag...,[ecefbf6e5a5967b21e19fc00d93af357],de3b561f5cce4c83bccb39180e362c97,4,5,23,28


In [59]:
report_df = pd.read_parquet(r"C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_community_reports.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 91


c:\Users\aamruthvenkatesh.s\Music\GraphRag\env\lib\site-packages\graphrag\query\indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
c:\Users\aamruthvenkatesh.s\Music\GraphRag\env\lib\site-packages\graphrag\query\indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,89,# The Gazette of India and Legal Publications\...,3,8.5,The Gazette of India and Legal Publications,The impact severity rating is high due to the ...,The community revolves around The Gazette of I...,[{'explanation': 'The Gazette of India is the ...,"{\n ""title"": ""The Gazette of India and Lega...",c90d155c-dcef-4d22-a3a3-df80dcb1beb7
1,90,# India's Legal and Judicial Framework\n\nThe ...,3,8.5,India's Legal and Judicial Framework,The impact severity rating is high due to the ...,The community revolves around the legal and ju...,[{'explanation': 'India serves as the central ...,"{\n ""title"": ""India's Legal and Judicial Fr...",847dd5e3-8638-437a-9459-40cc632a5c4b
2,58,# Accused Person and Legal Obligations\n\nThe ...,2,7.5,Accused Person and Legal Obligations,The impact severity rating is high due to the ...,The community revolves around the legal obliga...,[{'explanation': 'The accused person is the ce...,"{\n ""title"": ""Accused Person and Legal Obli...",475e6886-3c3d-454c-bee4-a6991ffe8944
3,59,"# Judicial and Correctional System: Court, Jai...",2,8.5,"Judicial and Correctional System: Court, Jail,...",The impact severity rating is high due to the ...,This community revolves around the judicial an...,[{'explanation': 'The Court is a pivotal entit...,"{\n ""title"": ""Judicial and Correctional Sys...",42f31208-7b62-4059-8bb2-690a6f59ceb8
4,60,# Offender and Legal Proceedings Community\n\n...,2,8.5,Offender and Legal Proceedings Community,The impact severity rating is high due to the ...,"The community revolves around the Offender, wh...",[{'explanation': 'The Court plays a crucial ro...,"{\n ""title"": ""Offender and Legal Proceeding...",2f217fb5-81d6-4e93-b775-f9ddd1cd1672


In [60]:
text_unit_df = pd.read_parquet(r"C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_text_units.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 147


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,ecefbf6e5a5967b21e19fc00d93af357,"# jftLVªh lañ Mhñ ,yñ—(,u)04@0007@2003—23 REGI...",1200,[fee1a13daa06e0032634b00b27951e41],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[58a8fa7f29e347bdb9689b70b065a779, fae3fe31deb..."
1,380bac9e6c010716fe963157a4eed21a,a Union territory to which the jurisdiction o...,1200,[fee1a13daa06e0032634b00b27951e41],"[e7ffaee9d31d4d3c96e04f911d0a8f9e, f7e11b0e297...","[4bc7dc91ede345dfb63d7d4f7ac3554f, 14f8ac195fd..."
2,6f0a4ba398c2c089d00f97b143611753,regulating the manner or place of investigati...,1200,[fee1a13daa06e0032634b00b27951e41],"[e7ffaee9d31d4d3c96e04f911d0a8f9e, 27f9fbe6ad8...","[0b2b815c9f834aaaac0c341097def9ba, 424ae71c560..."
3,ad97c615278d3bc561e6e7c82a6dc1f6,Judicial Magistrate of the first class in any...,1200,[fee1a13daa06e0032634b00b27951e41],"[e7ffaee9d31d4d3c96e04f911d0a8f9e, 27f9fbe6ad8...","[0b2b815c9f834aaaac0c341097def9ba, 586cf02da94..."
4,9d11b65b9c218276b034c36d84dc3b4e,"under this Sanhita on Executive Magistrates, ...",1200,[fee1a13daa06e0032634b00b27951e41],"[e7ffaee9d31d4d3c96e04f911d0a8f9e, 27f9fbe6ad8...","[6242e0c237a348908d0256ea790a0211, 1b06d3e53ff..."


In [61]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch the values
api_key = os.getenv("GRAPHRAG_API_KEY")
llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o-mini',
    # api_base='https://openai-ppcazure017-eastus.openai.azure.com/',
    api_version='2024-02-15-preview',
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    # api_base='https://openai-ppcazure017-eastus.openai.azure.com/',
    api_type=OpenaiApiType.OpenAI,
    api_version='2024-02-15-preview',
    model='text-embedding-3-small',
    # deployment_name=embedding_model,
    max_retries=20,
)

In [62]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [63]:
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 4_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [64]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [65]:
result = await search_engine.asearch("What will be the punishment and section Number for The suspect was caught stealing a bicycle from the street. The bicycle was locked, and the suspect broke the lock to take it away?")
print(result.response)

In the case of stealing a bicycle that was locked, the relevant section of the Bharatiya Nyaya Sanhita, 2023, would likely fall under the general provisions for theft. Specifically, the act of breaking a lock to steal the bicycle can be categorized as theft after preparation, which is addressed in Section 307 of the Bharatiya Nyaya Sanhita. This section outlines the punishment for theft after preparation made for causing death, hurt, or restraint in order to commit theft.

The punishment for this offense, as specified in Section 307, is rigorous imprisonment for a term that can extend up to 10 years, along with a fine [Data: Entities (644); Sources (109)]. 

Additionally, if the theft is considered to be committed in a dwelling house or involves other specific circumstances, different sections may apply, but based on the information provided, Section 307 is the most relevant for this scenario.


In [66]:
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 1. LLM tokens: 9617


In the case of stealing a bicycle that was locked, the relevant section of the Bharatiya Nyaya Sanhita, 2023, would likely fall under the provisions for theft. Specifically, the act of breaking a lock to steal the bicycle indicates a more serious offense due to the preparation involved in committing the theft.

### Relevant Sections and Punishments

1. **Theft**: The general act of stealing property, which includes bicycles, is punishable under various sections of the Bharatiya Nyaya Sanhita. The specific section that may apply here is:
   - **Section 307**: This section outlines the punishment for theft after preparation made for causing death, hurt, or restraint in order to commit theft. While this section is more severe, it reflects the seriousness of breaking a lock to facilitate the theft. The punishment under this section can include rigorous imprisonment for up to 10 years and a fine [Data: Entities (644); Sources (109)].

2. **Theft in a Dwelling House**: If the bicycle was considered to be in a dwelling house or a similar protected area, it could also fall under:
   - **Section 306**: This section deals with theft by a clerk or servant of property in possession of the master or employer, which may not directly apply here but indicates the seriousness of theft in protected areas [Data: Entities (564)].

### Conclusion

In summary, the suspect caught stealing a locked bicycle could face serious penalties under the Bharatiya Nyaya Sanhita, particularly under Section 307, which addresses theft involving preparation for causing restraint. The punishment could involve rigorous imprisonment for a significant term, reflecting the severity of the crime committed.

In [37]:
result.context_data["entities"].head()


,id,entity,description,number of relationships,in_context
0,643,SECTION 303(2),"Section 303(2) of the Bharatiya Nyaya Sanhita,...",1,True
1,644,SECTION 307,SECTION 307 of the Bharatiya Nyaya Sanhita out...,1,True
2,639,SECTION 309(2),Section 309(2) of the Bharatiya Nyaya Sanhita ...,1,True
3,640,SECTION 310(2),Section 310(2) of the Bharatiya Nyaya Sanhita ...,1,True
4,578,BELONGING TO A WANDERING GANG FOR THEFT,Belonging to a wandering gang of persons assoc...,1,True


In [38]:
result.context_data["relationships"].head()


,id,source,target,description,weight,rank,links,in_context
0,241,MAGISTRATE,THEFT,MAGISTRATE and THEFT are interconnected in the...,2.0,148,8,True
1,244,MAGISTRATE,ROBBERY,Magistrates have the authority to hear and dec...,2.0,146,8,True
2,245,MAGISTRATE,THEFT IN A DWELLING HOUSE,Magistrates have the authority to hear and dec...,1.0,145,8,True
3,246,MAGISTRATE,THEFT BY CLERK OR SERVANT,Magistrates have the authority to hear and dec...,1.0,145,8,True
4,247,MAGISTRATE,THEFT AFTER PREPARATION,Magistrates have the authority to hear and dec...,1.0,145,8,True


In [39]:
result.context_data["reports"].head()


,id,title,content
0,42,Bharatiya Nyaya Sanhita and Legal Framework in...,# Bharatiya Nyaya Sanhita and Legal Framework ...
1,42,Bharatiya Nyaya Sanhita and Legal Framework in...,# Bharatiya Nyaya Sanhita and Legal Framework ...


In [40]:
result.context_data["sources"].head()


,id,text
0,109,"to the committing of such theft, and\nthereby..."
1,108,.................................................
2,94,Non-bailable.|Any Magistrate.|\n|305|Theft in ...
3,60,feelings are intended to be wounded.|\n|Theft...


## Global Search

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

c:\Users\aamruthvenkatesh.s\Music\GraphRag\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch the values
api_key = os.getenv("GRAPHRAG_API_KEY")
llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o-mini',
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

In [3]:
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(r'C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_nodes.parquet')
entity_embedding_df = pd.read_parquet(r"C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_entities.parquet")
report_df = pd.read_parquet(r"C:\Users\aamruthvenkatesh.s\Music\GraphRag\ragtest\output\20240818-181527\artifacts\create_final_community_reports.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

c:\Users\aamruthvenkatesh.s\Music\GraphRag\env\lib\site-packages\graphrag\query\indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
c:\Users\aamruthvenkatesh.s\Music\GraphRag\env\lib\site-packages\graphrag\query\indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Total report count: 91
Report count after filtering by community level 2: 68


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,89,# The Gazette of India and Legal Publications\...,3,8.5,The Gazette of India and Legal Publications,The impact severity rating is high due to the ...,The community revolves around The Gazette of I...,[{'explanation': 'The Gazette of India is the ...,"{\n ""title"": ""The Gazette of India and Lega...",c90d155c-dcef-4d22-a3a3-df80dcb1beb7
1,90,# India's Legal and Judicial Framework\n\nThe ...,3,8.5,India's Legal and Judicial Framework,The impact severity rating is high due to the ...,The community revolves around the legal and ju...,[{'explanation': 'India serves as the central ...,"{\n ""title"": ""India's Legal and Judicial Fr...",847dd5e3-8638-437a-9459-40cc632a5c4b
2,58,# Accused Person and Legal Obligations\n\nThe ...,2,7.5,Accused Person and Legal Obligations,The impact severity rating is high due to the ...,The community revolves around the legal obliga...,[{'explanation': 'The accused person is the ce...,"{\n ""title"": ""Accused Person and Legal Obli...",475e6886-3c3d-454c-bee4-a6991ffe8944
3,59,"# Judicial and Correctional System: Court, Jai...",2,8.5,"Judicial and Correctional System: Court, Jail,...",The impact severity rating is high due to the ...,This community revolves around the judicial an...,[{'explanation': 'The Court is a pivotal entit...,"{\n ""title"": ""Judicial and Correctional Sys...",42f31208-7b62-4059-8bb2-690a6f59ceb8
4,60,# Offender and Legal Proceedings Community\n\n...,2,8.5,Offender and Legal Proceedings Community,The impact severity rating is high due to the ...,"The community revolves around the Offender, wh...",[{'explanation': 'The Court plays a crucial ro...,"{\n ""title"": ""Offender and Legal Proceeding...",2f217fb5-81d6-4e93-b775-f9ddd1cd1672


In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 4000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 4000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [8]:
result = await search_engine.asearch(
    "What will be the punishment and section Number for The suspect was caught stealing a bicycle from the street. The bicycle was locked, and the suspect broke the lock to take it away?"
)

print(result.response)

## Legal Implications of Bicycle Theft

The act of stealing a bicycle, particularly by breaking a lock, is classified as theft under the relevant legal frameworks. The punishment for such an offense may include imprisonment and/or fines, which are determined based on the severity of the crime and the value of the stolen property. The specific penalties are typically outlined in the Bharatiya Nyaya Sanhita, 2023, which addresses theft and related offenses [Data: Reports (77, 61, 57, 64, 54)].

### Additional Charges

In addition to the primary charge of theft, the act of breaking a lock may lead to further legal consequences. This could include additional charges related to property damage or destruction, as well as potential charges for house-trespass or mischief. However, the specific section numbers for these offenses are not detailed in the available data [Data: Reports (88, 85)].

### Section Numbers

While the legal provisions for theft are generally outlined in the Bharatiya Nyay

In [9]:
result.context_data["reports"]


,id,title,occurrence weight,content,rank
0,76,Gazette of India and Legal Framework,0.973333,# Gazette of India and Legal Framework\n\nThe ...,8.5
1,66,Court of Session and Related Judicial Entities...,0.626667,# Court of Session and Related Judicial Entiti...,8.5
2,74,State Government and Legal Framework in India,0.586667,# State Government and Legal Framework in Indi...,8.5
3,33,District Magistrate and Judicial System in India,0.480000,# District Magistrate and Judicial System in I...,8.5
4,80,Sanhita and Bharatiya Sakshya Adhiniyam Legal ...,0.333333,# Sanhita and Bharatiya Sakshya Adhiniyam Lega...,8.5
...,...,...,...,...,...
63,28,Supreme Court and Indian Judicial System,0.080000,# Supreme Court and Indian Judicial System\n\n...,9.0
64,45,Arrests and Associated Crimes,0.066667,# Arrests and Associated Crimes\n\nThe communi...,8.5
65,25,Legal Procedures and Appeals Community,0.066667,# Legal Procedures and Appeals Community\n\nTh...,7.5
66,62,Legal and Familial Contexts of Children,0.066667,# Legal and Familial Contexts of Children\n\nT...,7.5


In [10]:
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 6. LLM tokens: 58948
